In [13]:
!apt install mongodb > log

In [59]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [60]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.


In [138]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import time

In [139]:
client = MongoClient('localhost',27017)
db = client.vacancies_db
hh_vac = db.hh_vac

In [128]:
def hh_search(vacancy, page, only_new = 'yes', hh_vac = hh_vac):

    vacancy_data = []

    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)

    i = 0

    for p in range(page):
        html = requests.get(link, params=params, headers=headers).text
        parsed_html = bs(html,'lxml')
        vac_list = parsed_html.findAll('div', {'class':'vacancy-serp-item-body__main-info'})
        df = pd.DataFrame(columns=['name', 'salary_min', 'salary_max', 'salary_currency', 'url', 'site'])

        for vac in vac_list:
            vac_data = {}

            link = vac.find('a')
            if not link:
                url = 'nan'
                vac_data['name'] = ''
            else:
                vac_data['name'] = link.getText()
                url = link["href"].split('?')[0]
                vac_num = url.split('/')[4]

            salary = vac.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            
            if not salary:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                salary = salary.getText().replace(u'\u202f', u'')
        
                salary = re.split(r"\s", salary)
        
                if salary[0] == 'до':
                    salary_min = None
                    salary_max = int(salary[1])
                    salary_currency = salary[2]
                elif salary[0] == 'от':
                    salary_min = int(salary[1])
                    salary_max = None
                    salary_currency = salary[2]
                else:
                    salary_min = int(salary[0])
                    salary_max = int(salary[2])
                    salary_currency = salary[3]
        
            vac_data['salary_min'] = salary_min
            vac_data['salary_max'] = salary_max
            vac_data['salary_currency'] = salary_currency
            
            vac_data['url'] = url
            vac_data['vac_num'] = vac_num
            vac_data['site'] = 'hh.ru'

            vacancy_data.append(vac_data)
            df.append(vac_data, ignore_index = True)
            if only_new == 'yes':
                if find_in_db('hh', vac_num) == 0:
                    hh_vac.insert_one(vac_data)
            else:
                hh_vac.insert_one(vac_data)

            i += 1
        time.sleep(1)
    return(vacancy_data)


In [122]:
def find_in_db(site, vac_num):
  exists = 0
  if site == 'hh':
    exists = hh_vac.count_documents({'vac_num': vac_num})
  return exists


In [34]:
def search_vac(only_new = 'yes'):

  text = input('Введите вакансию: ')
  text = text.replace(' ', '+')
  page = int(input('Введите кол-во страниц: '))

  hh_search(text, page, only_new)

  df = pd.DataFrame.from_dict(hh_search(text, page))
  return df

In [140]:
print(search_vac('yes'))

Введите вакансию: Python
Введите кол-во страниц: 1
                                                 name  salary_min  salary_max  \
0            Ведущий инженер по тестированию (python)         NaN         NaN   
1                        Backend программист (Python)    120000.0         NaN   
2                                  Разработчик Python    200000.0    250000.0   
3                           Программист Python Junior         NaN         NaN   
4                                  Программист Python     45000.0         NaN   
5                             Junior Python developer     50000.0     90000.0   
6                    Python-разработчик (Django, DRF)     60000.0    220000.0   
7                                  Разработчик Python    200000.0    250000.0   
8                             Разработчик C# / Python    300000.0    500000.0   
9   Junior Python Backend Developer - разработчик ...     80000.0    120000.0   
10       Python Backend Developer / WEB - разработчик    3

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [136]:
def find_salary():
  salary = int(input('Введите минимальную зарплату: '))
  expensive_vacs = hh_vac.find({'$or': [{'salary_min':{'$gt':salary}}, {'salary_max':{'$gt':salary}}]})#.sort('salary_min')
  #hh_vac.drop()
  print('Вакансии с зарплатой выше чем ', salary)
  for item in expensive_vacs:
    pprint(item)

In [141]:
find_salary()

Введите минимальную зарплату: 250000
Вакансии с зарплатой выше чем  250000
{'_id': ObjectId('63f8c4c55c1592ff465d7b91'),
 'name': 'Разработчик C# / Python',
 'salary_currency': 'руб.',
 'salary_max': 500000,
 'salary_min': 300000,
 'site': 'hh.ru',
 'url': 'https://hh.ru/vacancy/71147348',
 'vac_num': '71147348'}
{'_id': ObjectId('63f8c4c55c1592ff465d7b93'),
 'name': 'Python Backend Developer / WEB - разработчик',
 'salary_currency': 'руб.',
 'salary_max': 450000,
 'salary_min': 300000,
 'site': 'hh.ru',
 'url': 'https://hh.ru/vacancy/72216524',
 'vac_num': '72216524'}
{'_id': ObjectId('63f8c4c55c1592ff465d7b9c'),
 'name': 'Lead Python developer',
 'salary_currency': 'руб.',
 'salary_max': 500000,
 'salary_min': None,
 'site': 'hh.ru',
 'url': 'https://hh.ru/vacancy/77337809',
 'vac_num': '77337809'}
